In [ ]:
# main function for the Monte Carlo Tree Search
def monte_carlo_tree_search(root):
	
	while resources_left(time, computational power):
		leaf = traverse(root) 
		simulation_result = rollout(leaf)
		backpropagate(leaf, simulation_result)
		
	return best_child(root)

# function for node traversal
def traverse(node):
	while fully_expanded(node):
		node = best_uct(node)
		
	# in case no children are present / node is terminal 
	return pick_unvisited(node.children) or node 

# function for the result of the simulation
def rollout(node):
	while non_terminal(node):
		node = rollout_policy(node)
	return result(node) 

# function for randomly selecting a child node
def rollout_policy(node):
	return pick_random(node.children)

# function for backpropagation
def backpropagate(node, result):
	if is_root(node) return
	node.stats = update_stats(node, result) 
	backpropagate(node.parent)

# function for selecting the best child
# node with highest number of visits
def best_child(node):
	pick child with highest number of visits


In [1]:
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # Represents a 3x3 board
        self.current_player = 'X'  # X always starts the game

    def is_winner(self, player):
        win_conditions = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Horizontal
            [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Vertical
            [0, 4, 8], [2, 4, 6]  # Diagonal
        ]
        for condition in win_conditions:
            if all(self.board[i] == player for i in condition):
                return True
        return False

    def is_draw(self):
        return ' ' not in self.board and not self.is_winner('X') and not self.is_winner('O')

    def make_move(self, index):
        if self.board[index] == ' ':
            self.board[index] = self.current_player
            self.current_player = 'O' if self.current_player == 'X' else 'X'
            return True
        return False

    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == ' ']

    def is_terminal(self):
        return self.is_winner('X') or self.is_winner('O') or self.is_draw()

    def __str__(self):
        return "\n".join(["|".join(self.board[i:i+3]) for i in range(0, 9, 3)])



In [3]:
import random
import math
from datetime import datetime, timedelta

class Node:
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = game_state
        self.parent = parent
        self.move = move
        self.children = []
        self.wins = 0
        self.visits = 0
        self.untried_moves = game_state.available_moves()

    def expand(self):
        move = self.untried_moves.pop()
        next_state = TicTacToe()
        next_state.board = self.game_state.board.copy()
        next_state.current_player = self.game_state.current_player
        next_state.make_move(move)
        child_node = Node(next_state, self, move)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        if self.game_state.current_player == result:
            self.wins += 1

    def is_fully_expanded(self):
        return len(self.untried_moves) == 0

    def best_child(self, c_param=1.41):
        choices_weights = [
            (child.wins / child.visits) + c_param * math.sqrt((2 * math.log(self.visits) / child.visits))
            for child in self.children
        ]
        return self.children[choices_weights.index(max(choices_weights))]

    def rollout_policy(self, possible_moves):
        return random.choice(possible_moves)

def monte_carlo_tree_search(root, max_seconds=3):
    end_time = datetime.now() + timedelta(seconds=max_seconds)
    while datetime.now() < end_time:
        leaf = traverse(root)
        simulation_result = rollout(leaf)
        backpropagate(leaf, simulation_result)

    return best_child(root, 0)

def traverse(node):
    while node.is_fully_expanded() and node.children:
        node = node.best_child()
    if not node.is_fully_expanded():
        return node.expand()
    return node

def rollout(node):
    current_rollout_state = TicTacToe()
    current_rollout_state.board = node.game_state.board.copy()
    current_rollout_state.current_player = node.game_state.current_player

    while not current_rollout_state.is_terminal():
        possible_moves = current_rollout_state.available_moves()
        if possible_moves:  # Ensures there are available moves to play
            move = node.rollout_policy(possible_moves)
            current_rollout_state.make_move(move)
        else:
            break

    if current_rollout_state.is_winner('X'):
        return 'X'
    if current_rollout_state.is_winner('O'):
        return 'O'
    return 'draw'

def best_child(node, c_param=1.41):
    # Adjusting to ensure there's at least one child to select
    if node.children:
        choices_weights = [
            (child.wins / child.visits) + c_param * math.sqrt((2 * math.log(node.visits) / child.visits))
            for child in node.children
        ]
        return node.children[choices_weights.index(max(choices_weights))]
    return node  # Fallback if no children exist



def backpropagate(node, result):
    while node is not None:
        node.update(result)
        node = node.parent

def best_child(node, exploration_value=0):
    best = max(node.children, key=lambda x: x.wins / x.visits + exploration_value)
    return best.move if best else None

# Ensure node is properly initialized and has moves to play
root = Node(TicTacToe())
if not root.is_fully_expanded():
    best_move = monte_carlo_tree_search(root)
    print("Best Move:", best_move)
else:
    print("No moves available or game is at a terminal state.")



Best Move: 1
